In [1]:
from util import *
from glob import glob

In [2]:
df = load_AOIs()
df

,Taranaki,AOI,SSP 4.5 (p50),SSP 4.5 (p83),SSP 8.5 (p50),SSP 8.5 (p83),Rate SSP 4.5 (p50),Rate SSP 4.5 (p83),Rate SSP 8.5 (p50),Rate SSP 8.5 (p83),match,match_score
7,NORTH,TongaporutuRiver,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110,TongapurutuRiverCliffs,93.750000
11,SOUTH,HangatahuaRiver_South,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110,HangatahuRiver_South,97.435897
21,SOUTH,Rahotu,0.58,0.78,0.84,1.10,0.0058,0.0078,0.0084,0.0110,Rahotu,100.000000
20,SOUTH,Pihama,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110,Pihama,100.000000
19,SOUTH,OpunakeBeach,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110,OpunakeBeachCliffs,100.000000
18,SOUTH,OhaweBeach,0.57,0.78,0.83,1.10,0.0057,0.0078,0.0083,0.0110,OhaweBeach,100.000000
17,SOUTH,Oeo,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110,Oeo,100.000000
16,SOUTH,Manutahi,0.57,0.78,0.83,1.10,0.0057,0.0078,0.0083,0.0110,Manutahi,100.000000
15,SOUTH,ManaBay,0.57,0.78,0.83,1.10,0.0057,0.0078,0.0083,0.0110,ManaBayCliffs,100.000000
14,SOUTH,KaupokonuiBeach,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110,KaupokonuiBeach,100.000000


In [3]:
site = df.match.sample(1).iloc[0]
site

'ManaBayCliffs'

In [4]:
gdf = gpd.read_file(f"Shapefiles/{site}_intersects.shp")
gdf.crs = 2193
gdf = enrich_df(gdf)
transect_metadata = get_transect_metadata(f"Shapefiles/{site}_TransectLines.shp")
if site == "ManaBayCliffs":
  print("Flipping")
  for k, v in transect_metadata.items():
    transect_metadata[k]["Azimuth"] = v["Azimuth"] + 180

Flipping


In [5]:
linear_models = fit(gdf, transect_metadata)
linear_models.loc[linear_models.slope > 0, "slope"] = pd.NA
rolled_slopes = linear_models.groupby("group").slope.rolling(10, min_periods=1).mean().dropna().reset_index(level=0)
linear_models.slope = rolled_slopes.slope
linear_models.dropna(inplace=True)
linear_models.index = linear_models.TransectID
linear_models

,TransectID,slope,intercept,group
TransectID,,,,
1,1,-0.222779,32.660453,0
2,2,-0.242045,29.421904,0
3,3,-0.299418,30.078610,0
4,4,-0.299532,28.908556,0
5,5,-0.269567,31.602443,0
...,...,...,...,...
636,636,-0.010913,30.747370,22
637,637,-0.016195,30.273036,22
639,639,-0.081485,28.457112,23


In [6]:
# Find the closest data point to 2000
gdf["years_since_2000"] = (gdf.Year - 2000).abs()
closest_to_2000 = gdf.loc[gdf.groupby("TransectID").years_since_2000.idxmin()]
closest_to_2000.set_index("TransectID", inplace=True)
closest_to_2000

,OBJECTID,TransOrder,BaselineID,ShorelineI,Distance,IntersectX,IntersectY,Uncertaint,AOI,geometry,Date,Year,YearsSinceBase,YearsUntilFuture,years_since_2000
TransectID,,,,,,,,,,,,,,,
1,2,1,1,05/04/1962,27.701682,1.724263e+06,5.598505e+06,2.411659,ManaBay,POINT (1724262.766 5598505.109),1962-04-05,1962,22.258727,137.741273,38
2,4,2,1,05/04/1962,23.605455,1.724273e+06,5.598504e+06,2.411659,ManaBay,POINT (1724273.483 5598503.724),1962-04-05,1962,22.258727,137.741273,38
3,6,3,1,05/04/1962,20.859829,1.724284e+06,5.598501e+06,2.411659,ManaBay,POINT (1724283.533 5598501.165),1962-04-05,1962,22.258727,137.741273,38
4,8,4,1,05/04/1962,22.233757,1.724292e+06,5.598495e+06,2.411659,ManaBay,POINT (1724291.545 5598495.025),1962-04-05,1962,22.258727,137.741273,38
5,10,5,1,05/04/1962,28.270124,1.724297e+06,5.598485e+06,2.411659,ManaBay,POINT (1724297.251 5598484.834),1962-04-05,1962,22.258727,137.741273,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,3347,637,3,26/07/2007,26.641123,1.730611e+06,5.593965e+06,1.633952,ManaBay,POINT (1730611.402 5593964.757),2007-07-26,2007,67.564682,92.435318,7
638,3350,638,3,26/07/2007,22.947984,1.730620e+06,5.593959e+06,1.633952,ManaBay,POINT (1730620.291 5593958.874),2007-07-26,2007,67.564682,92.435318,7
639,3353,639,3,26/07/2007,20.403636,1.730628e+06,5.593952e+06,1.633952,ManaBay,POINT (1730628.255 5593952.312),2007-07-26,2007,67.564682,92.435318,7


In [7]:
closest_to_2000.Year.value_counts()

2007    517
2012     34
1962     13
2017     13
Name: Year, dtype: int64

In [8]:
# Predict to the closest year to 2000
linear_models["FUTURE_YEAR"] = closest_to_2000.Year
linear_models

,TransectID,slope,intercept,group,FUTURE_YEAR
TransectID,,,,,
1,1,-0.222779,32.660453,0,1962
2,2,-0.242045,29.421904,0,1962
3,3,-0.299418,30.078610,0,1962
4,4,-0.299532,28.908556,0,1962
5,5,-0.269567,31.602443,0,1962
...,...,...,...,...,...
636,636,-0.010913,30.747370,22,2007
637,637,-0.016195,30.273036,22,2007
639,639,-0.081485,28.457112,23,2007


In [9]:
results = predict(gdf, linear_models, transect_metadata)
results

,TransectID,BaselineID,group,Year,ocean_point,linear_model_point,linear_model_distance,sqrt_model_point,sqrt_model_distance,BH_model_point,BH_model_distance,Sunamura_model_point,Sunamura_model_distance
0,1.0,1,0.0,1962,POINT (1724015.4914425344 5598070.5336709935),POINT (1724278.946518806 5598533.545537542),32.718092,POINT (1724278.9817532038 5598533.6074606),32.789338,POINT (1724278.974932633 5598533.595473717),32.775546,POINT (1724278.955270141 5598533.560917666),32.735788
1,2.0,1,0.0,1962,POINT (1724026.2087396842 5598069.14873204),POINT (1724288.064660726 5598529.3501465805),29.484528,POINT (1724288.1029422258 5598529.417424801),29.561935,POINT (1724288.0930745532 5598529.400082757),29.541982,POINT (1724288.0734100018 5598529.365523087),29.502219
2,3.0,1,0.0,1962,POINT (1724036.258231437 5598066.58984261),POINT (1724298.4462665136 5598527.374934418),30.156078,POINT (1724298.4936221 5598527.458159987),30.251833,POINT (1724298.4746803406 5598527.424870594),30.213532,POINT (1724298.455009657 5598527.390300147),30.173757
3,4.0,1,0.0,1962,POINT (1724044.2703504052 5598060.450474325),POINT (1724305.879751441 5598520.218639715),28.986053,POINT (1724305.9271250262 5598520.301896915),29.081845,POINT (1724305.908165268 5598520.26857589),29.043507,POINT (1724305.8884945724 5598520.234005421),29.003732
4,5.0,1,0.0,1962,POINT (1724049.9768976246 5598050.258614339),POINT (1724312.914722903 5598512.361432743),31.672187,POINT (1724312.9573573084 5598512.436361023),31.758396,POINT (1724312.9431367298 5598512.41136892),31.729641,POINT (1724312.9234692368 5598512.376804079),31.689873
...,...,...,...,...,...,...,...,...,...,...,...,...,...
563,636.0,3,22.0,2007,POINT (1730198.0325473628 5593674.280275961),POINT (1730626.2560650462 5593988.036966946),30.865936,POINT (1730626.3742842476 5593988.123585414),31.012492,POINT (1730628.2022023562 5593989.462889571),33.278550,POINT (1730626.8570178698 5593988.477281318),31.610934
564,637.0,3,22.0,2007,POINT (1730205.4949799979 5593667.351208605),POINT (1730633.3821632317 5593980.861469443),30.448985,POINT (1730633.5575974411 5593980.99000899),30.666469,POINT (1730635.3283005415 5593982.287392069),32.861598,POINT (1730633.9830773894 5593981.301755485),31.193934
565,639.0,3,23.0,2007,POINT (1730221.8764661436 5593654.560503963),POINT (1730648.8710290326 5593967.416747462),29.342409,POINT (1730649.7537364294 5593968.063501647),30.436696,POINT (1730650.8171663426 5593968.842670088),31.755023,POINT (1730649.471465215 5593967.856683295),30.086766
566,640.0,3,23.0,2007,POINT (1730225.7444246176 5593644.997304726),POINT (1730653.7203756508 5593958.572605176),30.559029,POINT (1730654.4697645647 5593959.121677742),31.488042,POINT (1730655.6665129608 5593959.998527801),32.971643,POINT (1730654.3209019299 5593959.012607021),31.303498


In [10]:
results.index = results.TransectID.astype(int)
results

,TransectID,BaselineID,group,Year,ocean_point,linear_model_point,linear_model_distance,sqrt_model_point,sqrt_model_distance,BH_model_point,BH_model_distance,Sunamura_model_point,Sunamura_model_distance
TransectID,,,,,,,,,,,,,
1,1.0,1,0.0,1962,POINT (1724015.4914425344 5598070.5336709935),POINT (1724278.946518806 5598533.545537542),32.718092,POINT (1724278.9817532038 5598533.6074606),32.789338,POINT (1724278.974932633 5598533.595473717),32.775546,POINT (1724278.955270141 5598533.560917666),32.735788
2,2.0,1,0.0,1962,POINT (1724026.2087396842 5598069.14873204),POINT (1724288.064660726 5598529.3501465805),29.484528,POINT (1724288.1029422258 5598529.417424801),29.561935,POINT (1724288.0930745532 5598529.400082757),29.541982,POINT (1724288.0734100018 5598529.365523087),29.502219
3,3.0,1,0.0,1962,POINT (1724036.258231437 5598066.58984261),POINT (1724298.4462665136 5598527.374934418),30.156078,POINT (1724298.4936221 5598527.458159987),30.251833,POINT (1724298.4746803406 5598527.424870594),30.213532,POINT (1724298.455009657 5598527.390300147),30.173757
4,4.0,1,0.0,1962,POINT (1724044.2703504052 5598060.450474325),POINT (1724305.879751441 5598520.218639715),28.986053,POINT (1724305.9271250262 5598520.301896915),29.081845,POINT (1724305.908165268 5598520.26857589),29.043507,POINT (1724305.8884945724 5598520.234005421),29.003732
5,5.0,1,0.0,1962,POINT (1724049.9768976246 5598050.258614339),POINT (1724312.914722903 5598512.361432743),31.672187,POINT (1724312.9573573084 5598512.436361023),31.758396,POINT (1724312.9431367298 5598512.41136892),31.729641,POINT (1724312.9234692368 5598512.376804079),31.689873
...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,636.0,3,22.0,2007,POINT (1730198.0325473628 5593674.280275961),POINT (1730626.2560650462 5593988.036966946),30.865936,POINT (1730626.3742842476 5593988.123585414),31.012492,POINT (1730628.2022023562 5593989.462889571),33.278550,POINT (1730626.8570178698 5593988.477281318),31.610934
637,637.0,3,22.0,2007,POINT (1730205.4949799979 5593667.351208605),POINT (1730633.3821632317 5593980.861469443),30.448985,POINT (1730633.5575974411 5593980.99000899),30.666469,POINT (1730635.3283005415 5593982.287392069),32.861598,POINT (1730633.9830773894 5593981.301755485),31.193934
639,639.0,3,23.0,2007,POINT (1730221.8764661436 5593654.560503963),POINT (1730648.8710290326 5593967.416747462),29.342409,POINT (1730649.7537364294 5593968.063501647),30.436696,POINT (1730650.8171663426 5593968.842670088),31.755023,POINT (1730649.471465215 5593967.856683295),30.086766


In [11]:
results["True"] = closest_to_2000.geometry
results

,TransectID,BaselineID,group,Year,ocean_point,linear_model_point,linear_model_distance,sqrt_model_point,sqrt_model_distance,BH_model_point,BH_model_distance,Sunamura_model_point,Sunamura_model_distance,True
TransectID,,,,,,,,,,,,,,
1,1.0,1,0.0,1962,POINT (1724015.4914425344 5598070.5336709935),POINT (1724278.946518806 5598533.545537542),32.718092,POINT (1724278.9817532038 5598533.6074606),32.789338,POINT (1724278.974932633 5598533.595473717),32.775546,POINT (1724278.955270141 5598533.560917666),32.735788,POINT (1724262.766 5598505.109)
2,2.0,1,0.0,1962,POINT (1724026.2087396842 5598069.14873204),POINT (1724288.064660726 5598529.3501465805),29.484528,POINT (1724288.1029422258 5598529.417424801),29.561935,POINT (1724288.0930745532 5598529.400082757),29.541982,POINT (1724288.0734100018 5598529.365523087),29.502219,POINT (1724273.483 5598503.724)
3,3.0,1,0.0,1962,POINT (1724036.258231437 5598066.58984261),POINT (1724298.4462665136 5598527.374934418),30.156078,POINT (1724298.4936221 5598527.458159987),30.251833,POINT (1724298.4746803406 5598527.424870594),30.213532,POINT (1724298.455009657 5598527.390300147),30.173757,POINT (1724283.533 5598501.165)
4,4.0,1,0.0,1962,POINT (1724044.2703504052 5598060.450474325),POINT (1724305.879751441 5598520.218639715),28.986053,POINT (1724305.9271250262 5598520.301896915),29.081845,POINT (1724305.908165268 5598520.26857589),29.043507,POINT (1724305.8884945724 5598520.234005421),29.003732,POINT (1724291.545 5598495.025)
5,5.0,1,0.0,1962,POINT (1724049.9768976246 5598050.258614339),POINT (1724312.914722903 5598512.361432743),31.672187,POINT (1724312.9573573084 5598512.436361023),31.758396,POINT (1724312.9431367298 5598512.41136892),31.729641,POINT (1724312.9234692368 5598512.376804079),31.689873,POINT (1724297.251 5598484.834)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,636.0,3,22.0,2007,POINT (1730198.0325473628 5593674.280275961),POINT (1730626.2560650462 5593988.036966946),30.865936,POINT (1730626.3742842476 5593988.123585414),31.012492,POINT (1730628.2022023562 5593989.462889571),33.278550,POINT (1730626.8570178698 5593988.477281318),31.610934,POINT (1730603.187 5593971.135)
637,637.0,3,22.0,2007,POINT (1730205.4949799979 5593667.351208605),POINT (1730633.3821632317 5593980.861469443),30.448985,POINT (1730633.5575974411 5593980.99000899),30.666469,POINT (1730635.3283005415 5593982.287392069),32.861598,POINT (1730633.9830773894 5593981.301755485),31.193934,POINT (1730611.402 5593964.757)
639,639.0,3,23.0,2007,POINT (1730221.8764661436 5593654.560503963),POINT (1730648.8710290326 5593967.416747462),29.342409,POINT (1730649.7537364294 5593968.063501647),30.436696,POINT (1730650.8171663426 5593968.842670088),31.755023,POINT (1730649.471465215 5593967.856683295),30.086766,POINT (1730628.255 5593952.312)


In [12]:
m = closest_to_2000.explore(color="green", tiles="Esri.WorldImagery")
for model in SUPPORTED_MODELS:
  gpd.GeoDataFrame(results[f"{model}_model_distance"], geometry=results[f"{model}_model_point"], crs=2193).explore(color="blue", m=m)
m

In [13]:
for model in SUPPORTED_MODELS:
  print(model)
  print(gpd.GeoSeries(results[f"{model}_model_point"], crs=2193).distance(results["True"]).describe())

linear
count    568.000000
mean      65.627097
std       34.985711
min       24.988473
25%       37.137263
50%       57.817627
75%       79.375278
max      241.574325
dtype: float64
sqrt
count    568.000000
mean      70.003212
std       38.515127
min       25.047145
25%       38.880427
50%       61.684916
75%       85.283480
max      256.649632
dtype: float64
BH
count    568.000000
mean      68.494673
std       35.074902
min       27.401087
25%       40.142931
50%       60.971975
75%       82.135385
max      243.618351
dtype: float64
Sunamura
count    568.000000
mean      66.509743
std       35.009697
min       25.733515
25%       38.110786
50%       58.788527
75%       80.344672
max      242.195401
dtype: float64
